# Предобработка

## Первоначальная обработка (категории)

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [3]:
data=pd.read_csv("../Data/Petitions.csv")

In [4]:
data.head()

,id,public_petition_text,reason_category
0,3168490,снег на дороге,Благоустройство
1,3219678,очистить кабельный киоск от рекламы,Благоустройство
2,2963920,"Просим убрать все деревья и кустарники, которы...",Благоустройство
3,3374910,Неудовлетворительное состояние парадной - надп...,Содержание МКД
4,3336285,Граффити,Благоустройство


In [5]:
data=data.drop(columns=["id"])

In [6]:
data["reason_category"].unique()

array(['Благоустройство', 'Содержание МКД',
       'Незаконная информационная и (или) рекламная конструкция', 'Фасад',
       'Водоснабжение', 'Нарушение правил пользования общим имуществом',
       'Повреждения или неисправность элементов уличной инфраструктуры',
       'Кровля', 'Состояние рекламных или информационных конструкций',
       'Нарушение порядка пользования общим имуществом', 'Подвалы',
       'Водоотведение', 'Санитарное состояние', 'Центральное отопление',
       'Незаконная реализация товаров с торгового оборудования (прилавок, ящик, с земли)'],
      dtype=object)

In [7]:
for category in data["reason_category"].unique():
    print(category,data["reason_category"].isin([category]).sum())

Благоустройство 34769
Содержание МКД 14461
Незаконная информационная и (или) рекламная конструкция 1831
Фасад 1483
Водоснабжение 809
Нарушение правил пользования общим имуществом 2170
Повреждения или неисправность элементов уличной инфраструктуры 1164
Кровля 825
Состояние рекламных или информационных конструкций 673
Нарушение порядка пользования общим имуществом 243
Подвалы 254
Водоотведение 238
Санитарное состояние 434
Центральное отопление 284
Незаконная реализация товаров с торгового оборудования (прилавок, ящик, с земли) 251


<h3> Посмотрим какие категорие похожи по содержанию, чтобы объединить их

In [8]:
for category in data["reason_category"].unique():
    print("\n------------------------",category)
    category_data=data.loc[data["reason_category"]==category].reset_index()
    for i in range(5):
        print ("\n",i+1,"жалоба:",category_data["public_petition_text"][i])


------------------------ Благоустройство

 1 жалоба: снег на дороге

 2 жалоба: очистить кабельный киоск от рекламы

 3 жалоба: Просим убрать все деревья и кустарники, которые вышли за пределы газона на пешеходную зону, начиная от подъезда №13 до подъезда №15 (фасад дома со стороны ул. Наличной).

 4 жалоба: Граффити

 5 жалоба: Мусор

------------------------ Содержание МКД

 1 жалоба: Неудовлетворительное состояние парадной - надписи на двери 2 этажа

 2 жалоба: Уборка не производится, на лестнице очень грязно. На всех этажах, вплоть до 5-го.
Звонок в ЖКС#2 5.04.2021 не дал результатов.

 3 жалоба: Отсутствует освещение на лестничной площадке между 6 и 7 этажами в парадной № 2

 4 жалоба: Проблема с регулярным вывозом мусора

 5 жалоба: Ланское шоссе д12к1 парадная 4
поврежден почтовый ящик, прошу заменить

------------------------ Незаконная информационная и (или) рекламная конструкция

 1 жалоба: Необходимо проверить законность установки вывески на фасаде МКД по адресу проспект Не

<h3>Объелиняем некоторые категории, дабы обобщить и уменьшить число классов

In [9]:
data.loc[data["reason_category"].isin(["Водоснабжение","Водоотведение","Центральное отопление"]),
        "reason_category"]="Нарушение подачи и отведения горячей и холодной воды"
data.loc[data["reason_category"]=="Санитарное состояние","reason_category"] = "Содержание МКД"
data.loc[data["reason_category"].isin(["Незаконная информационная и (или) рекламная конструкция",
                        "Незаконная реализация товаров с торгового оборудования (прилавок, ящик, с земли)",
                           "Состояние рекламных или информационных конструкций" ]),"reason_category"] = "Проблемы с торговлей, рекламой, информационными стендами"
data.loc[data["reason_category"] == "Повреждения или неисправность элементов уличной инфраструктуры","reason_category"] = "Благоустройство"
data.loc[data["reason_category"] == "Нарушение правил пользования общим имуществом","reason_category"] = "Нарушение порядка пользования общим имуществом"
data.loc[data["reason_category"].isin(["Кровля",
                            "Фасад","Подвалы"]),"reason_category"]="Состояние МКД"

In [10]:
for category in data["reason_category"].unique():
    print(category,data["reason_category"].isin([category]).sum())

Благоустройство 35933
Содержание МКД 14895
Проблемы с торговлей, рекламой, информационными стендами 2755
Состояние МКД 2562
Нарушение подачи и отведения горячей и холодной воды 1331
Нарушение порядка пользования общим имуществом 2413


## токенезация

In [11]:
import nltk
nltk.download('punkt')
nltk.download('punkt_tab')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\ov00v\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package punkt_tab to
[nltk_data]     C:\Users\ov00v\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


True

In [12]:
from nltk.tokenize import sent_tokenize, word_tokenize
import re

In [13]:
data.head(20)

,public_petition_text,reason_category
0,снег на дороге,Благоустройство
1,очистить кабельный киоск от рекламы,Благоустройство
2,"Просим убрать все деревья и кустарники, которы...",Благоустройство
3,Неудовлетворительное состояние парадной - надп...,Содержание МКД
4,Граффити,Благоустройство
5,Необходимо проверить законность установки выве...,"Проблемы с торговлей, рекламой, информационным..."
6,"Уборка не производится, на лестнице очень гряз...",Содержание МКД
7,Мусор,Благоустройство
8,Отсутствует освещение на лестничной площадке м...,Содержание МКД
9,"Зачем было делать благоустройство, если никто ...",Благоустройство


In [14]:
data["public_petition_text"]=data["public_petition_text"].apply(lambda x:
        [word_tokenize(re.sub('[^а-яА-Я ]+', '', sent).lower()) for sent in sent_tokenize(x)])

In [15]:
data.head(20)

,public_petition_text,reason_category
0,"[[снег, на, дороге]]",Благоустройство
1,"[[очистить, кабельный, киоск, от, рекламы]]",Благоустройство
2,"[[просим, убрать, все, деревья, и, кустарники,...",Благоустройство
3,"[[неудовлетворительное, состояние, парадной, н...",Содержание МКД
4,[[граффити]],Благоустройство
5,"[[необходимо, проверить, законность, установки...","Проблемы с торговлей, рекламой, информационным..."
6,"[[уборка, не, производится, на, лестнице, очен...",Содержание МКД
7,[[мусор]],Благоустройство
8,"[[отсутствует, освещение, на, лестничной, площ...",Содержание МКД
9,"[[зачем, было, делать, благоустройство, если, ...",Благоустройство


## Лемматизация

In [16]:
!pip install pymorphy3

You should consider upgrading via the 'D:\NLP\nlp\Scripts\python.exe -m pip install --upgrade pip' command.


In [17]:
from pymorphy3 import MorphAnalyzer
morph = MorphAnalyzer()

In [18]:
data["public_petition_text"]=data["public_petition_text"].apply(lambda x: [[morph.normal_forms(w)[0] for w in words] for words in x])

In [19]:
data.head(20)

,public_petition_text,reason_category
0,"[[снег, на, дорога]]",Благоустройство
1,"[[очистить, кабельный, киоск, от, реклама]]",Благоустройство
2,"[[просить, убрать, всё, дерево, и, кустарник, ...",Благоустройство
3,"[[неудовлетворительный, состояние, парадный, н...",Содержание МКД
4,[[граффити]],Благоустройство
5,"[[необходимо, проверить, законность, установка...","Проблемы с торговлей, рекламой, информационным..."
6,"[[уборка, не, производиться, на, лестница, оче...",Содержание МКД
7,[[мусор]],Благоустройство
8,"[[отсутствовать, освещение, на, лестничный, пл...",Содержание МКД
9,"[[зачем, быть, делать, благоустройство, если, ...",Благоустройство


## Удаление стоп-слов

In [20]:
nltk.download('stopwords')
from nltk.corpus import stopwords
stopwords = stopwords.words('russian')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\ov00v\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [21]:
data["public_petition_text"]=data["public_petition_text"].apply(lambda x: [[w for w in words if w not in stopwords] for words in x])

In [22]:
data.head(20)

,public_petition_text,reason_category
0,"[[снег, дорога]]",Благоустройство
1,"[[очистить, кабельный, киоск, реклама]]",Благоустройство
2,"[[просить, убрать, всё, дерево, кустарник, кот...",Благоустройство
3,"[[неудовлетворительный, состояние, парадный, н...",Содержание МКД
4,[[граффити]],Благоустройство
5,"[[необходимо, проверить, законность, установка...","Проблемы с торговлей, рекламой, информационным..."
6,"[[уборка, производиться, лестница, очень, гряз...",Содержание МКД
7,[[мусор]],Благоустройство
8,"[[отсутствовать, освещение, лестничный, площад...",Содержание МКД
9,"[[делать, благоустройство, никто, убирать, мус...",Благоустройство


## Удаление редко встречаемых слов

In [23]:
import pandas as pd
from collections import Counter

n = 20

all_words = [word for sublist in data['public_petition_text'] for inner_list in sublist for word in inner_list]
word_count = Counter(all_words)
def filter_rare_words(nested_words):
    return [[word if word_count[word] >= n else "RWT" for word in inner_list] for inner_list in nested_words]
data['public_petition_text'] = data['public_petition_text'].apply(filter_rare_words)
word_count["RWT"]=0

In [24]:
data.head(-20)

,public_petition_text,reason_category
0,"[[снег, дорога]]",Благоустройство
1,"[[очистить, кабельный, киоск, реклама]]",Благоустройство
2,"[[просить, убрать, всё, дерево, кустарник, кот...",Благоустройство
3,"[[неудовлетворительный, состояние, парадный, н...",Содержание МКД
4,[[граффити]],Благоустройство
...,...,...
59864,"[[газон, мусор, бутылка]]",Благоустройство
59865,"[[снег, дорога]]",Благоустройство
59866,"[[], [], [поменять, радиатор, отопление, это, ...",Содержание МКД
59867,"[[течь, подавать, стояк, RWT, этаж], [попытка,...",Нарушение подачи и отведения горячей и холодно...


<h3> Чистка пустых предложений

In [25]:
def delete_empty_sent(corpus):
    new_corpus=[sent for sent in corpus if len(sent)>=1]
    return new_corpus if len(new_corpus)>=1 else None
data['public_petition_text'] = data['public_petition_text'].apply(delete_empty_sent)
data.head(-20)

,public_petition_text,reason_category
0,"[[снег, дорога]]",Благоустройство
1,"[[очистить, кабельный, киоск, реклама]]",Благоустройство
2,"[[просить, убрать, всё, дерево, кустарник, кот...",Благоустройство
3,"[[неудовлетворительный, состояние, парадный, н...",Содержание МКД
4,[[граффити]],Благоустройство
...,...,...
59864,"[[газон, мусор, бутылка]]",Благоустройство
59865,"[[снег, дорога]]",Благоустройство
59866,"[[поменять, радиатор, отопление, это, большой,...",Содержание МКД
59867,"[[течь, подавать, стояк, RWT, этаж], [попытка,...",Нарушение подачи и отведения горячей и холодно...


In [26]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 59889 entries, 0 to 59888
Data columns (total 2 columns):
 #   Column                Non-Null Count  Dtype 
---  ------                --------------  ----- 
 0   public_petition_text  59889 non-null  object
 1   reason_category       59889 non-null  object
dtypes: object(2)
memory usage: 935.9+ KB


## Сохраняем предобработанные данные

In [27]:
data.to_json("../Data/Petitions_prepared.json")

In [3]:
data=data.drop(columns=['reason_category'])


NameError: name 'data' is not defined

In [139]:
data.head()

,public_petition_text
0,"[[снег, дорога]]"
1,"[[очистить, кабельный, киоск, реклама]]"
2,"[[просить, убрать, всё, дерево, кустарник, кот..."
3,"[[неудовлетворительный, состояние, парадный, н..."
4,[[граффити]]


# Создание матрицы контекстных эмбедингов

## Инициализация

In [4]:
context_embeding_matrix = pd.DataFrame(0,index=word_count.keys(), columns=word_count.keys())

NameError: name 'word_count' is not defined

In [141]:
context_embeding_matrix

,снег,дорога,очистить,кабельный,киоск,реклама,просить,убрать,всё,дерево,...,водоотвод,платформа,зловонный,передний,упаковка,допустить,неужели,моховой,войти,перепланировка
снег,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
дорога,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
очистить,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
кабельный,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
киоск,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
допустить,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
неужели,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
моховой,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
войти,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


## Заполнение

In [142]:
for corpus in data['public_petition_text']:
    for sentence in corpus:
        if len(sentence)>=2:
            for w in range(len(sentence)-1):
                context_embeding_matrix.loc[sentence[w+1],sentence[w]]+=1
                context_embeding_matrix.loc[sentence[w],sentence[w+1]]+=1

In [143]:
for word in context_embeding_matrix.columns:
     context_embeding_matrix.loc[word,word]=0

In [144]:
context_embeding_matrix.head(20)

,снег,дорога,очистить,кабельный,киоск,реклама,просить,убрать,всё,дерево,...,водоотвод,платформа,зловонный,передний,упаковка,допустить,неужели,моховой,войти,перепланировка
снег,0,985,98,0,0,0,2,654,11,0,...,0,0,0,0,0,0,0,0,1,0
дорога,985,0,31,0,0,0,3,31,6,1,...,0,0,0,0,0,0,0,0,0,0
очистить,98,31,0,2,0,57,299,7,9,4,...,0,0,0,0,0,0,0,0,0,0
кабельный,0,0,2,0,39,1,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
киоск,0,0,0,39,0,2,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
реклама,0,0,57,1,2,0,3,163,3,29,...,0,0,0,0,0,0,0,0,0,0
просить,2,3,299,0,0,3,0,1004,3,3,...,0,0,0,0,0,0,0,0,0,0
убрать,654,31,7,1,0,163,1004,0,50,15,...,0,0,0,0,0,0,0,0,0,0
всё,11,6,9,0,0,3,3,50,0,4,...,0,0,0,0,0,0,0,0,0,1
дерево,0,1,4,0,0,29,3,15,4,0,...,0,0,0,0,0,0,0,0,0,0


In [145]:
context_embeding_matrix.to_csv("../Data/context_embeding_matrix.csv")

## Уменьшение размерности

In [2]:
context_embeding_matrix=pd.read_csv("../Data/context_embeding_matrix.csv").drop(columns=["Unnamed: 0"])

In [3]:
from sklearn.decomposition import PCA

In [4]:
context_embeding_matrix_pca=PCA(n_components=10).fit_transform(context_embeding_matrix)

In [5]:
context_embeding_matrix

,снег,дорога,очистить,кабельный,киоск,реклама,просить,убрать,всё,дерево,...,водоотвод,платформа,зловонный,передний,упаковка,допустить,неужели,моховой,войти,перепланировка
0,0,985,98,0,0,0,2,654,11,0,...,0,0,0,0,0,0,0,0,1,0
1,985,0,31,0,0,0,3,31,6,1,...,0,0,0,0,0,0,0,0,0,0
2,98,31,0,2,0,57,299,7,9,4,...,0,0,0,0,0,0,0,0,0,0
3,0,0,2,0,39,1,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,39,0,2,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2710,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2711,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2712,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2713,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [6]:
context_embeding_matrix_pca.shape

(2715, 10)

In [138]:
context_embeding_matrix_pca=pd.DataFrame(context_embeding_matrix_pca,index=context_embeding_matrix.columns)
context_embeding_matrix_pca

,0,1,2,3,4,5,6,7,8,9
снег,421.637810,269.514368,-442.351344,552.733883,329.807717,6.788361,25.150241,6.037114,-67.897076,-75.236715
дорога,279.061053,20.305615,-287.364289,-349.339906,189.101388,1.616951,-48.212080,28.655819,16.838172,136.958496
очистить,144.691036,-16.929233,-158.665935,-218.038709,71.702104,5.535839,169.611275,-106.244019,-95.814789,-154.034091
кабельный,-21.497143,-11.509115,0.973695,1.025578,-3.341731,-0.808950,9.427517,5.558923,5.208359,-4.838700
киоск,-19.640134,-7.421501,0.849053,0.161158,-3.524194,-1.050412,6.298134,1.966427,3.942430,-2.906290
...,...,...,...,...,...,...,...,...,...,...
допустить,-23.226130,-9.288755,2.059868,1.783853,0.408438,-1.592012,-1.707047,-1.266402,0.652537,-0.451560
неужели,-24.958122,-12.685664,2.282893,2.154184,0.710202,-1.722710,-1.738685,-0.918717,0.795400,-0.642384
моховой,-25.234228,-18.115702,2.907962,3.082568,-0.438942,-2.549568,-2.721850,-0.752263,1.207247,-1.152072
войти,-23.552848,-11.745653,3.924105,1.796576,1.886891,-3.032075,-1.480059,-1.261149,1.012477,-0.560953


# Word2Vec

## Подготовка данных для обучение модели

<h3> Создание словарей

In [8]:
word2idx=dict(zip(context_embeding_matrix.columns,range(1,len(context_embeding_matrix.columns)+1)))
word2idx["out_of_sentence"]=0
idx2word={v: k for k, v in word2idx.items()}
vocab_size = len(word2idx)
embedding_dim = 10
context_size = 2

In [9]:
sents=pd.read_json("../Data/Petitions_prepared.json")["public_petition_text"]
sents

0                                         [[снег, дорога]]
1                  [[очистить, кабельный, киоск, реклама]]
2        [[просить, убрать, всё, дерево, кустарник, кот...
3        [[неудовлетворительный, состояние, парадный, н...
4                                             [[граффити]]
                               ...                        
59884                     [[просить, закрасить, граффити]]
59885    [[просить, отремонтировать, пешеходный, дорожка]]
59886    [[необходимо, демонтировать, незаконный, рекла...
59887    [[очень, RWT, ветер, металлический, часть, окн...
59888    [[проблема, незаконный, надписиобъявление, сте...
Name: public_petition_text, Length: 59889, dtype: object

In [10]:
import torch
from torch.utils.data import Dataset, DataLoader

<h3> Создание датасета

In [11]:
class Word2VecDataset(Dataset):
    def __init__(self,sents, word2idx, context_size):
        self.data = []
        for corpus in sents:
            for sentence in corpus:
                indices = [word2idx[word] for word in sentence]
                for i in range(len(indices)):
                    context = []
                    # Получение контекста (слов вокруг целевого)
                    for j in range(i - context_size, i + context_size + 1):
                        if j != i:
                            if 0 <= j < len(indices):
                                context.append(indices[j])
                            else: context.append(0)
                    target = indices[i]
                    self.data.append((context, target))

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        context, target = self.data[idx]
        return torch.tensor(context, dtype=torch.long), torch.tensor(target, dtype=torch.long)

dataset = Word2VecDataset(sents, word2idx, context_size)
dataloader = DataLoader(dataset, batch_size=20, shuffle=True)

## Создание модели

In [14]:
import torch.nn as nn

class CBOW(nn.Module):
    def __init__(self, vocab_size, embedding_dim):
        super(CBOW, self).__init__()
        self.embeddings = nn.Embedding(vocab_size, embedding_dim)
        self.fc = nn.Linear(embedding_dim, vocab_size)

    def forward(self, context):
        # Получаем векторы для контекстных слов
        embeds = self.embeddings(context)
        # Берем среднее значение векторов контекста
        context_vector = embeds.mean(dim=1)
        output = self.fc(context_vector)
        return output

model = CBOW(vocab_size, embedding_dim)
loss_function = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=0.01)


In [15]:
torch.cuda.is_available()

True

In [16]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)

## Обучение модели

In [20]:
num_epochs = 100
previous_loss=np.inf
for epoch in range(num_epochs):
    total_loss = 0
    for context, target in dataloader:
        context, target = context.to(device), target.to(device)
        optimizer.zero_grad()  # Обнуляем градиенты
        output = model(context)  # Прямой проход
        loss = loss_function(output, target)  # Вычисляем потери
        loss.backward()  # Обратный проход
        optimizer.step()  # Обновляем веса
        total_loss += loss.item()
    
    if (epoch + 1) % 2 == 0:
        print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {total_loss:.4f}')
    if total_loss>=previous_loss:
        print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {total_loss:.4f}')
        break
    previous_loss=total_loss

Epoch [2/100], Loss: 180274.1593
Epoch [4/100], Loss: 177092.0552
Epoch [6/100], Loss: 174326.8715
Epoch [8/100], Loss: 171870.4246
Epoch [10/100], Loss: 169729.9976
Epoch [12/100], Loss: 167863.6965
Epoch [14/100], Loss: 166220.3253
Epoch [16/100], Loss: 164764.8380
Epoch [18/100], Loss: 163472.5521
Epoch [20/100], Loss: 162308.1420
Epoch [22/100], Loss: 161250.7491
Epoch [24/100], Loss: 160283.1323
Epoch [26/100], Loss: 159394.7317
Epoch [28/100], Loss: 158570.0120
Epoch [30/100], Loss: 157803.3127
Epoch [32/100], Loss: 157086.5630
Epoch [34/100], Loss: 156414.8921
Epoch [36/100], Loss: 155783.3534
Epoch [38/100], Loss: 155189.3660
Epoch [40/100], Loss: 154628.0702
Epoch [42/100], Loss: 154097.1654
Epoch [44/100], Loss: 153594.2754
Epoch [46/100], Loss: 153115.4533
Epoch [48/100], Loss: 152658.8948
Epoch [50/100], Loss: 152226.4171
Epoch [52/100], Loss: 151810.1205
Epoch [54/100], Loss: 151413.2219
Epoch [56/100], Loss: 151031.4682
Epoch [58/100], Loss: 150667.2002
Epoch [60/100], Lo

In [21]:
torch.save(model,"..\Models\my_own_CBOW")

## Демонстрация результов модели

In [33]:
# Получение векторов слов
word_embeddings = model.embeddings.weight.data.cpu().numpy()

# Пример вывода вектора для слова "снег"
word_idx = word2idx['убрать']
print(f'Вектор для слова "убрать": {word_embeddings[word_idx]}')


Вектор для слова "убрать": [-2.0538993  -0.3502988  -1.2166536   0.86898345  0.6302003   3.6286325
  2.2869518  -0.33058253 -2.900491    1.7537504 ]


In [42]:
import torch.nn.functional as F

In [132]:
context1 = torch.tensor([[word2idx["помочь"],word2idx["холодный"],word2idx["труба"],word2idx["конец"]]]).to(device)#word2idx["комитет"]
context2 = torch.tensor([[word2idx["RWT"],word2idx["тсж"],word2idx["RWT"],word2idx["рубль"]]]).to(device)#word2idx["полгода"]
# Получаем предсказание
logits1 = model(context1)
logits2 = model(context2)
# Применяем softmax для получения вероятностей
probabilities1 = F.softmax(logits1, dim=1)
probabilities2 = F.softmax(logits2, dim=1)
predicted_index1 = torch.argmax(probabilities1).item()
predicted_index2 = torch.argmax(probabilities2).item()
print(f'1 предсказанное слово: {idx2word[predicted_index1]}')
print(f'2 предсказанное слово: {idx2word[predicted_index2]}')

1 предсказанное слово: вода
2 предсказанное слово: суворовский


# Подготовка к отображению в Projector

In [136]:
pd.DataFrame(context_embeding_matrix_pca,index=context_embeding_matrix.columns)

,0,1,2,3,4,5,6,7,8,9
снег,421.637810,269.514368,-442.351344,552.733883,329.807717,6.788361,25.150241,6.037114,-67.897076,-75.236715
дорога,279.061053,20.305615,-287.364289,-349.339906,189.101388,1.616951,-48.212080,28.655819,16.838172,136.958496
очистить,144.691036,-16.929233,-158.665935,-218.038709,71.702104,5.535839,169.611275,-106.244019,-95.814789,-154.034091
кабельный,-21.497143,-11.509115,0.973695,1.025578,-3.341731,-0.808950,9.427517,5.558923,5.208359,-4.838700
киоск,-19.640134,-7.421501,0.849053,0.161158,-3.524194,-1.050412,6.298134,1.966427,3.942430,-2.906290
...,...,...,...,...,...,...,...,...,...,...
допустить,-23.226130,-9.288755,2.059868,1.783853,0.408438,-1.592012,-1.707047,-1.266402,0.652537,-0.451560
неужели,-24.958122,-12.685664,2.282893,2.154184,0.710202,-1.722710,-1.738685,-0.918717,0.795400,-0.642384
моховой,-25.234228,-18.115702,2.907962,3.082568,-0.438942,-2.549568,-2.721850,-0.752263,1.207247,-1.152072
войти,-23.552848,-11.745653,3.924105,1.796576,1.886891,-3.032075,-1.480059,-1.261149,1.012477,-0.560953


In [176]:
cem_indexis=pd.DataFrame(context_embeding_matrix_pca.index,index=context_embeding_matrix_pca.index)

In [178]:
context_embeding_matrix_pca.to_csv('..\Data\context_embeding_matrix_pca.tsv', sep='\t',header=False,index=False)
cem_indexis.to_csv('..\Data\cem_indexis.tsv', sep='\t')

In [179]:
embeding_matrix=pd.DataFrame(word_embeddings)
embeding_matrix=embeding_matrix.rename(index=lambda x:idx2word[x])
embeding_matrix

,0,1,2,3,4,5,6,7,8,9
out_of_sentence,-0.993693,0.659434,-1.238412,0.808247,0.923010,0.661832,1.464584,4.194146,-1.496444,2.731762
снег,-0.914271,-1.560739,4.899178,0.149349,4.276020,2.356515,-1.425284,1.913858,2.370657,2.047482
дорога,-0.588915,0.958551,-1.955188,4.315921,1.591163,1.450587,-1.033556,-1.982314,1.587972,2.571156
очистить,-2.402273,-0.767705,-0.322375,-1.219082,-0.041576,4.350973,2.148028,-0.974114,-1.658627,0.868369
кабельный,-0.233527,0.489035,-0.494179,-0.886290,0.489337,-0.197116,0.138816,0.617295,-1.770010,0.316113
...,...,...,...,...,...,...,...,...,...,...
допустить,0.705622,-0.270528,1.601545,1.909672,0.917147,1.316008,0.531082,0.472924,0.040428,-0.285764
неужели,-1.084575,0.877520,-0.034049,1.268097,-0.646995,1.364565,0.314063,1.346161,-0.578789,-0.884693
моховой,-0.351579,-1.824729,0.457157,0.361208,-0.866471,0.173058,-1.043274,-0.117404,0.527315,0.310002
войти,0.952945,0.037924,2.023438,1.068429,0.905199,-0.468186,0.423652,-1.114097,0.435680,0.247058


In [180]:
emb_indexis=pd.DataFrame(embeding_matrix.index,index=embeding_matrix.index)

In [1]:
embeding_matrix.to_csv('..\Data\embeding_matrix.tsv', sep='\t',header=False,index=False)
emb_indexis.to_csv('..\Data\emb_indexis.tsv', sep='\t')

NameError: name 'embeding_matrix' is not defined

# Доп задание (классификация)

## Подготовка данных

In [185]:
diabetic_train= pd.read_csv('..\Data\diabetic_prepared.csv')
diabetic_test=pd.read_csv('..\Data\diabetic_test.csv')

In [186]:
diabetic_test.head()

,Diabetes_012,HighBP,HighChol,CholCheck,BMI,Smoker,Stroke,HeartDiseaseorAttack,PhysActivity,Fruits,...,AnyHealthcare,NoDocbcCost,GenHlth,MentHlth,PhysHlth,DiffWalk,Sex,Age,Education,Income
0,0,1,1,1,31,1,0,0,1,0,...,0,0,3,0,2,1,0,10,6,6
1,0,1,1,1,24,0,0,0,1,1,...,1,0,3,0,0,0,0,13,5,6
2,0,0,0,1,24,0,0,0,1,1,...,1,0,2,6,4,0,0,4,6,8
3,0,0,0,1,30,0,0,0,0,1,...,1,0,2,3,0,0,1,9,5,7
4,0,1,0,1,27,1,0,0,1,0,...,1,0,1,0,0,0,1,12,5,3


In [188]:
from torch.utils.data import TensorDataset

In [217]:
diabetic_train_dataset=TensorDataset(torch.tensor(diabetic_train.drop(columns=["Diabetes_012"]).values,dtype=torch.float32),
                                     torch.tensor(diabetic_train["Diabetes_012"].values,dtype=torch.float32))
diabetic_data_loader = DataLoader(diabetic_train_dataset, batch_size=10, shuffle=True)

## Создание и обучение модели

In [306]:
class CLASS(nn.Module):
    def __init__(self, input_size):
        super(CLASS, self).__init__()
        self.l1 = nn.Linear(input_size, 10)
        self.l2 = nn.Linear(10, 1)
    def forward(self, input_tensor):
        result = self.l1(input_tensor)
        output = self.l2(result)
        return output.squeeze()

In [307]:
diab_model = CLASS(21)
loss_function_diab = nn.BCEWithLogitsLoss()
optimizer_diab = torch.optim.SGD(diab_model.parameters(), lr=0.01)

In [316]:
num_epochs = 100
previous_loss=np.inf
diab_model.train()
for epoch in range(num_epochs):
    total_loss = 0
    for features, target in diabetic_data_loader:
        optimizer_diab.zero_grad()  # Обнуляем градиенты
        output = diab_model(features)  # Прямой проход
        loss = loss_function_diab(output, target)  # Вычисляем потери
        loss.backward()  # Обратный проход
        optimizer.step()  # Обновляем веса
        total_loss += loss.item()
    
    if (epoch + 1) % 2 == 0:
        print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {total_loss:.4f}')
    if total_loss>=previous_loss:
        print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {total_loss:.4f}')
        break
    previous_loss=total_loss

Epoch [2/100], Loss: 8921.0707
Epoch [3/100], Loss: 8921.1030


In [310]:
diab_model.eval()

CLASS(
  (l1): Linear(in_features=21, out_features=10, bias=True)
  (l2): Linear(in_features=10, out_features=1, bias=True)
)

## Тестирование

In [311]:
X_test=torch.tensor(diabetic_test.drop(columns=["Diabetes_012"]).values,dtype=torch.float32)
y_test=diabetic_test["Diabetes_012"].values
y_test

array([0, 0, 0, ..., 0, 1, 0])

In [321]:
test_outputs = diab_model(X_test)
predicted_probs = torch.sigmoid(test_outputs)  # Применяем сигмоиду для получения вероятностей
#predicted = (predicted_probs > 0.5).float()     # Преобразуем вероятности в классы
#accuracy = (predicted.view(-1) == torch.tensor(y_test).view(-1)).sum().item() / y_test.size * 100
#print(f'Точность модели на тестовом наборе: {accuracy:.2f}%')

In [323]:
result=np.rint(np.array(predicted_probs.detach()))
result

array([0., 0., 0., ..., 0., 0., 0.], dtype=float32)

In [324]:
from sklearn.metrics import  classification_report

In [325]:
print(classification_report(result,y_test))

              precision    recall  f1-score   support

         0.0       0.92      0.84      0.88     46283
         1.0       0.11      0.20      0.14      4453

    accuracy                           0.79     50736
   macro avg       0.51      0.52      0.51     50736
weighted avg       0.85      0.79      0.81     50736

